In [1]:
%load_ext autoreload
%autoreload 2

# 创建文档
from docx import Document

# 其他
from pyautoword import *
import pandas as pd
# to_markdown需要安装一个包tabulate

In [2]:
# 变量
xxmc = '西安工业大学'
year = '2022'
path = 'E:/桌面/test/'
# 额外分组，比如分学历，分师范生：
other_groupby = 'xl'

# 取数

In [3]:
# 获取派遣数据
conn = get_yjy_db('dw_database')

In [4]:
sql =f'''
    SELECT * FROM dw_s_employment_auto
    where xxmc = '{xxmc}' and substring(bynd,1,4) = '{year}'
    '''
data = pd.read_sql(sql=sql, con=conn)
data.head(1)

,xxdm,xxmc,bynd,yx,zydm,zy,bj,xm,xh,sfzhm,...,dim_company_property,dim_industry_category,dim_scale,dim_registered_capital,dim_dwmc_province,dim_dwmc_city,jylbfl,dwhy_c,sjly,xxmc_c
0,10702,西安工业大学,2022,建筑工程学院,082802,城乡规划,None,张家颖,17070206131,610111199910161540,...,,,,,,,签就业协议形式就业,建筑业,国库,None


# 获取分组表格并排序

In [11]:
%aimport pyautoword
string = 'zy'
data1 = get_group_order_table(data,string,other_groupby)

|     | zy                       | 研究生_num   | 研究生_proportion   | 本科_num   | 本科_proportion   |   num | proportion   |
|----:|:-------------------------|:-------------|:--------------------|:-----------|:------------------|------:|:-------------|
|   0 | 中国古代文学             | 2            | 0.04%               | -          | -                 |     2 | 0.04%        |
|   1 | 中国现当代文学           | 3            | 0.06%               | -          | -                 |     3 | 0.06%        |
|   2 | 中国近现代史基本问题研究 | 8            | 0.15%               | -          | -                 |     8 | 0.15%        |
|   3 | 书法学                   | -            | -                   | 50         | 0.93%             |    50 | 0.93%        |
|   4 | 产业经济学               | 2            | 0.04%               | -          | -                 |     2 | 0.04%        |
|   5 | 产品设计                 | -            | -                   | 105        | 1.96%             |   105 | 1.96%        |
|   6 | 人力资源管理         

请输入排序规则：s
未在表格中找到专业字段


In [7]:
dict_sf = {}
list_sf = ['河北省',
'香港特别行政区',
'天津市',
'山西省',
'内蒙古自治区',
'四川省',
'海南省',
'江西省',
'北京市',
'黑龙江省',
'云南省',
'西藏自治区',
'浙江省',
'广西壮族自治区',
'辽宁省',
'澳门特别行政区',
'福建省',
'安徽省',
'青海省',
'贵州省',
'吉林省',
'湖南省',
'陕西省',
'台湾省',
'山东省',
'江苏省',
'重庆市',
'广东省',
'湖北省',
'甘肃省',
'新疆维吾尔自治区',
'宁夏回族自治区',
'上海市',
'河南省']
for i in list_sf:
    dict_sf[i] = i.strip('省市自治区壮族维吾尔回族特别行政')

In [8]:
dict_sf

{'河北省': '河北',
 '香港特别行政区': '香港',
 '天津市': '天津',
 '山西省': '山西',
 '内蒙古自治区': '内蒙古',
 '四川省': '四川',
 '海南省': '海南',
 '江西省': '江西',
 '北京市': '北京',
 '黑龙江省': '黑龙江',
 '云南省': '云南',
 '西藏自治区': '西藏',
 '浙江省': '浙江',
 '广西壮族自治区': '广西',
 '辽宁省': '辽宁',
 '澳门特别行政区': '澳门',
 '福建省': '福建',
 '安徽省': '安徽',
 '青海省': '青海',
 '贵州省': '贵州',
 '吉林省': '吉林',
 '湖南省': '湖南',
 '陕西省': '陕西',
 '台湾省': '台湾',
 '山东省': '山东',
 '江苏省': '江苏',
 '重庆市': '重庆',
 '广东省': '广东',
 '湖北省': '湖北',
 '甘肃省': '甘肃',
 '新疆维吾尔自治区': '新疆',
 '宁夏回族自治区': '宁夏',
 '上海市': '上海',
 '河南省': '河南'}

In [9]:
def picture_china_map(data,picture_name,label_fontsize=13,color = get_lighter_color('#FFFFFF',color_1)[0]):
    data_zip = [*zip(data.index.tolist(),data)]#构建省份和对应统计人数的列表,地图数据传入的指定格式
    for i in range(len(data_zip)):
        data_zip[i]=[data_zip[i][0].strip('省市自治区壮族维吾尔回族特别行政'),data_zip[i][1]]
        data_zip[i][1]=i+1
    max_ = data_zip[len(data_zip)-1][1]
    a= (
    Map()
    .add("",data_zip,"china",is_map_symbol_show = False,is_roam = False,#数据传入,地图类型设置:中国地图"china"
         name_map=dict_sf,
         label_opts=opts.LabelOpts(formatter= '{b}',font_size=label_fontsize,font_family = '宋体'),#标签设置
         itemstyle_opts=opts.ItemStyleOpts(border_color = '#A6A6A6')) #图形的描边颜色
    .set_global_opts(
         visualmap_opts=opts.VisualMapOpts(is_show=False,max_ = max_,range_color = color))#颜色渐变设置
    )
    a = (
         Grid(init_opts = opts.InitOpts(height = '700px',width='1000px'))  #设置图形大小,高度:height,宽度:width
        .add(a, grid_opts=opts.GridOpts(pos_left='35%'))#位置调整
    )
    return make_snapshot(snapshot,a.render(), pic_path+f'{picture_name}.png',pixel_ratio = 5),a.render_notebook()

NameError: name 'get_lighter_color' is not defined

In [ ]:
raise

# 百分比四舍五入

In [ ]:
sum100([2,3,4])